In [ ]:
import numpy as np
import pandas as pd
import math
import copy
from PIL import Image
import cv2
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.neighbors import KernelDensity
from scipy.stats import norm
from bisect import bisect_left
import tensorflow as tf
from tensorflow import keras as keras
from keras import Input, Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv2D, Conv1D, Flatten, MaxPooling2D, concatenate, Add
from keras.layers import BatchNormalization, Dropout, SeparableConv2D, AvgPool2D
from keras import Input
from keras import optimizers

In [ ]:
def scale_data(X_train, X_test):
    scaler = MinMaxScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test

In [ ]:
def define_CNN_model(conv_layers, filters, size, pad, pool_func, pool_size, pool_stride):
    global my_lr
    
    model = Sequential()

    for layer in range(conv_layers):
        if pad==True:
            model.add(Conv2D(filters, size, activation=None, padding='same'))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            if pool_func=='max':
                model.add(MaxPooling2D((pool_size,pool_size), strides=pool_stride, padding='same'))
            elif pool_func=='average':
                model.add(AvgPool2D((pool_size,pool_size), strides=pool_stride, padding='same'))
        elif pad==False:
            model.add(Conv2D(filters, size, activation=None, padding='valid'))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            if pool_func=='max':
                model.add(MaxPooling2D((pool_size,pool_size), strides=pool_stride, padding='valid'))
            elif pool_func=='average':
                model.add(AvgPool2D((pool_size,pool_size), strides=pool_stride, padding='valid'))
    
    model.add(Flatten())
    model.add(Dropout(0.05))
    model.add(Dense(1, activation='linear'))

    my_opt = keras.optimizers.SGD(learning_rate = my_lr)
    
    model.compile(optimizer=my_opt, loss='mse', metrics=['mse'])
    
    return model 

In [ ]:
def define_model(hidden_layers, neurons, nonlinearity):
    global my_lr
    #initiate model
    model = Sequential()
    #create hidden layers
    for num_layers in range(hidden_layers):
        model.add(Dense(neurons, nonlinearity))
    #output layer
    model.add(Dense(1, 'linear'))

    #compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=my_lr), loss=['mse'], metrics=['mse'])

    return model

In [ ]:
def run_bootstrap(X_train, y_train, X_test, y_test, resamples):
    global layers, nodes, activation, size, pooling, padding, filters, pool_size, pool_stride
    global my_rng, my_epochs, my_batch, dataset, CC
    
    indices = np.zeros(((len(X_train),resamples)))
    oob_indices = {k: [] for k in range(resamples)}
  
    for b in range(resamples):
        indices[:, b] = resample(np.arange(len(X_train)), replace=True, n_samples=len(X_train), random_state = my_rng)
        oob_indices[b] = np.setdiff1d(np.arange(len(X_train)), indices[:,b],assume_unique=True)

    test_preds = np.zeros((len(X_test),resamples))
    errors = np.zeros(resamples)
    for b in range(resamples):
        #finding right datasets
        resampled_X = tf.convert_to_tensor(X_train[indices[:,b].astype(int)])
        resampled_y = tf.convert_to_tensor(y_train[indices[:,b].astype(int)])
        oob_X = tf.convert_to_tensor(X_train[oob_indices[b].astype(int)])
        oob_y = y_train[oob_indices[b].astype(int)]
        #define and fit model
        if dataset == 'RotNIST':
            model_b = define_CNN_model(conv_layers=layers, filters=filters, kernel_size=size, 
                                       pad = padding, pool_func=pooling, pool_size=pool_size, pool_stride=pool_stride) 
        else:
            model_b = define_model(hidden_layers = layers, neurons = nodes, nonlinearity = activation)
        model_b.fit(resampled_X, resampled_y, epochs=my_epochs, batch_size=my_batch, verbose=0)
        #estimating data noise 
        oob_preds_b = copy.deepcopy(model_b.predict(oob_X, batch_size=len(oob_X)))
        model_b_errors = np.array(oob_y).flatten() - np.array(oob_preds_b).flatten()
        model_b_rmse = np.sqrt(np.sum(model_b_errors**2)/(len(model_b_errors)-1))
        errors[b] = copy.deepcopy(model_b_rmse)
        #test preds
        test_preds_b = copy.deepcopy(model_b.predict(tf.convert_to_tensor(X_test), batch_size=len(X_test)))
        test_preds[:,b] = copy.deepcopy(test_preds_b.flatten())
        if (b+1) % 10 == 0: 
            print("{} models done!".format(b+1))
             
    data_noise = np.mean(errors)     
    
    ensemble_preds = np.mean(test_preds, axis=1)
    ensemble_errors = y_test.flatten() - ensemble_preds.flatten()
    rmse_k = np.sqrt(np.sum(ensemble_errors**2)/np.size(ensemble_errors))
    ensemble_std = np.std(test_preds, axis=1)

    boot_bounds = np.zeros((len(X_test),5))
    boot_bounds[:,0] = ensemble_preds + norm.ppf(0.5-CC/2) * np.sqrt(ensemble_std**2 + data_noise**2)
    boot_bounds[:,1] = ensemble_preds + norm.ppf(0.5+CC/2) * np.sqrt(ensemble_std**2 + data_noise**2)
    boot_bounds[:,2] = y_test.flatten()
    boot_bounds[:,3] = (y_test.flatten() - boot_bounds[:,0])*(boot_bounds[:,1] - y_test.flatten()) > 0
    boot_bounds[:,4] = boot_bounds[:,1] - boot_bounds[:,0]
    
    return boot_bounds

In [ ]:
def run_perc_bootstrap(X_train, y_train, X_test, y_test, resamples):
    global layers, activation, nodes, size, pooling, padding, filters, pool_size, pool_stride
    global my_rng, my_epochs, my_batch, dataset, CC
    
    indices = np.zeros(((len(X_train),resamples)))
    oob_indices = {k: [] for k in range(resamples)}
  
    for b in range(resamples):
        indices[:, b] = resample(np.arange(len(X_train)), replace=True, n_samples=len(X_train), random_state = my_rng)
        oob_indices[b] = np.setdiff1d(np.arange(len(X_train)), indices[:,b],assume_unique=True)

    test_preds = np.zeros((len(X_test),resamples))
    errors = np.zeros(resamples)
    for b in range(resamples):
        #finding right datasets
        resampled_X = tf.convert_to_tensor(X_train[indices[:,b].astype(int)])
        resampled_y = tf.convert_to_tensor(y_train[indices[:,b].astype(int)])
        oob_X = tf.convert_to_tensor(X_train[oob_indices[b].astype(int)])
        oob_y = y_train[oob_indices[b].astype(int)]
        #define and fit model
        if dataset == 'RotNIST':
            model_b = define_CNN_model(conv_layers=layers, filters=filters, kernel_size=size, 
                                       pad = padding, pool_func=pooling, pool_size=pool_size, pool_stride=pool_stride) 
        else:
            model_b = define_model(layers, nodes, activation)
        model_b.fit(resampled_X, resampled_y, epochs=my_epochs, batch_size=my_batch, verbose=0)
        #estimating data noise 
        oob_preds_b = copy.deepcopy(model_b.predict(oob_X, batch_size=len(oob_X)))
        model_b_errors = np.array(oob_y).flatten() - np.array(oob_preds_b).flatten()
        random_errors = resample(model_b_errors, replace=True, n_samples=len(X_test), random_state=my_rng).flatten()
        #test preds
        test_preds_b = model_b.predict(X_test, batch_size=len(X_test)).flatten()
        test_preds[:,b] = test_preds_b + random_errors
        if (b+1) % 10 == 0: 
            print("{} models done!".format(b+1))

    boot_bounds = np.zeros((len(X_test),5))
    boot_bounds[:,0] = np.percentile(test_preds, (0.5 - CC/2), axis=1)
    boot_bounds[:,1] = np.percentile(test_preds, (0.5 + CC/2), axis=1)
    boot_bounds[:,2] = y_test.flatten()
    boot_bounds[:,3] = (y_test.flatten() - boot_bounds[:,0])*(boot_bounds[:,1] - y_test.flatten()) > 0
    boot_bounds[:,4] = boot_bounds[:,1] - boot_bounds[:,0]
    
    return boot_bounds

In [ ]:
def run_full_conformal(X_train, y_train, X_test, y_test):
    global grid_fineness, layers, nodes, activation, size, pooling, padding, filters, pool_size, pool_stride
    global my_epochs, my_batch, trial_bandwidths, search_range, dataset, CC

    n = len(X_train)
    results = np.zeros((len(X_test),4,len(trial_bandwidths)))
    
    #train a network on the original training set... use predictions of test set to center PIs
    if dataset == 'RotNIST':
        model = define_CNN_model(conv_layers=layers, filters=filters, kernel_size=size, 
                                 pad = padding, pool_func=pooling, pool_size=pool_size, pool_stride=pool_stride) 
    else:
        model = define_model(layers, nodes, activation)
    model.fit(x = tf.convert_to_tensor(X_train),
              y = tf.convert_to_tensor(y_train),
              epochs=my_epochs,
              batch_size=my_batch,
              verbose=0)
    original_y_preds = model.predict(tf.convert_to_tensor(X_test)).flatten()
    
    pi = np.zeros((len(X_test),grid_fineness, len(trial_bandwidths)))
    candidates = np.zeros((len(X_test),grid_fineness))
    for obs in X_test.index:        
        #establish the search grid of candidate target values using the predicted y value and search range
        search_grid = np.linspace(start = original_y_preds[obs]-0.5*search_range,
                                  stop = original_y_preds[obs]+0.5*search_range,
                                  num=grid_fineness)
        candidates[obs,:] = search_grid
        #for each obs in test set, append it to the training feature set
        x = X_test.iloc[obs,:].values
        temp_X_train = X_train.copy(deep=True)
        temp_X_train.loc[len(temp_X_train.index)] = x

        #for each trial value y in the search grid, append it to the training target values
        #then fit the model 
        #then take residuals
        for trial, y in enumerate(search_grid):
            temp_y_train = np.concatenate((y_train, np.array([y])))
            if dataset == 'RotNIST':
                model = define_CNN_model(conv_layers=layers, filters=filters, kernel_size=size, 
                                         pad = padding, pool_func=pooling, pool_size=pool_size, pool_stride=pool_stride) 
            else:
                model = define_model(layers, nodes, activation)
            model.fit(x = tf.convert_to_tensor(temp_X_train),
                      y = tf.convert_to_tensor(temp_y_train),
                      epochs=my_epochs,
                      batch_size=my_batch,
                      verbose=0)
            #predict target values for original training set and calc resids
            preds_y_train = model.predict(tf.convert_to_tensor(temp_X_train), batch_size=len(temp_X_train))
            preds_y_train = preds_y_train
            resid = temp_y_train.flatten() - preds_y_train.flatten()
            
            for h, my_bandwidth in enumerate(trial_bandwidths):
                if my_bandwidth == 0:
                    calibration_scores = np.abs(resids[:-1])
                    critical_score = np.abs(resids[-1])
                else:
                    kde = KernelDensity(bandwidth=my_bandwidth).fit(resids.reshape(-1,1))
                    calibration_scores = -1 * kde.score_samples(resids.reshape(-1,1))
                    critical_score = -1 * kde.score_samples([[resids[-1]]])
                #now compare the residuals from the original training set to that of the trial y value
                pi[obs,trial,h] = (1 + np.sum(calibration_scores <= critical_score)) / (1 + len(calibration_scores))
                
    full_conformal = np.zeros((len(y_test),4,len(trial_bandwidths))) 
    full_conformal[:,1,:] = y_test.reshape(-1,1)
    full_conformal[:,0,:] = candidates.mean(axis=1).reshape(-1,1)

    for obs in range(len(y_test)):
        y_true = y_test[obs]

        for h in range(len(trial_bandwidths)):
            try:
                pred_region = pi[obs,:,h] < 0.95
                full_conformal[seed,obs,1,h] = search_range / grid_fineness * pred_region.sum()
                upper_arg = np.argwhere(candidates[obs,:] <= y_true).flatten().max()
                lower_arg = np.argwhere(candidates[obs,:] > y_true).flatten().min()
                full_conformal[seed,obs,2,h] = pred_region[lower_arg] * pred_region[upper_arg]
            except:
                full_conformal[seed,obs,2,h] = 0 
    
    return full_conformal

In [ ]:
def split_conformal(X_train, y_train, X_test, y_test):
    global activation, layers, nodes, size, pooling, padding, filters, pool_size, pool_stride
    global my_rng, my_epochs, my_batch, my_lr, grid_fineness, search_range, dataset, trial_bandwidths, CC
        
    #create proper training and calibration sets
    X_proper_train, X_calibration, y_proper_train, y_calibration = train_test_split(X_train, y_train,
                                                                                    test_size=0.5,
                                                                                    random_state=my_rng)
    
    #fit model to proper training data
    if dataset == 'RotNIST':
        model = define_CNN_model(conv_layers=layers, filters=filters, kernel_size=size, 
                                 pad = padding, pool_func=pooling, pool_size=pool_size, pool_stride=pool_stride) 
    else:
        model = define_model(layers, nodes, activation)
    model.fit(tf.convert_to_tensor(X_proper_train), 
              tf.convert_to_tensor(y_proper_train),
              epochs=my_epochs, 
              batch_size=my_batch, 
              verbose=0)

    #predict values in test set 
    test_preds = model.predict(tf.convert_to_tensor(X_test), batch_size=len(X_test)).flatten()

    #identify calibration sets and estimate residuals
    y_cal_preds = model.predict(tf.convert_to_tensor(X_calibration), batch_size=len(X_calibration))
    residuals = y_calibration.values.flatten() - y_cal_preds.flatten()
    
    #fit KDE distributions on augmented residual sets and compute p-value of potential residual values
    search_grid = np.linspace(0-search_range/2,
                              0+search_range/2,
                              grid_fineness)
    p_values = np.zeros((len(search_grid),len(trial_bandwidths)))
    for trial, resid_trial in enumerate(search_grid):
        aug_resids = np.concatenate([residuals, np.array([resid_trial])])
        for bandwidth, my_bandwidth in enumerate(trial_bandwidths):
            if my_bandwidth == 0:
                conform_scores = np.abs(aug_resids)
            else:
                kde = KernelDensity(bandwidth=my_bandwidth).fit(aug_resids.reshape(-1,1))
                conform_scores = -kde.score_samples(aug_resids.reshape(-1,1))
            p_values[trial,bandwidth] = np.sum(conform_scores <= conform_scores[-1]) / np.size(conform_scores)
    
    #initialize results array
    results = np.zeros((len(y_test),4,len(trial_bandwidths)))
    results[:,1,:] = y_test.reshape(-1,1)
    results[:,0,:] = test_preds.reshape(-1,1)
    
    #loop over each test value and bandwidth to find and evaluate prediction regions
    for h, my_bandwidth in enumerate(trial_bandwidths):
        mean_p_values = p_values[:,h]
        pred_region = mean_p_values < CC
        #width
        results[:,3,h] = np.sum(pred_region) / grid_fineness * search_range
        for test_obs, y_true in enumerate(y_test):
            critical_resid = y_true - test_preds[test_obs]
            try:
                lower_arg = np.argwhere(search_grid < critical_resid).flatten().max()
                upper_arg = np.argwhere(search_grid >= critical_resid).flatten().min()
                #in bounds?
                results[test_obs,2,h] = pred_region[lower_arg] * pred_region[upper_arg]
            except:
                #in bounds? 
                results[test_obs,2,h] = 0
        
    return results

In [ ]:
def cross_conformal(X_train, y_train, X_test, y_test, folds):
    global my_rng, activation, layers, nodes, size, pooling, padding, filters, pool_size, pool_stride
    global my_epochs, my_batch, my_lr, grid_fineness, search_range, dataset, trial_bandwidths, CC
     
    #initiate models
    model_dict = {}
    for k in range(folds):
        if dataset == 'RotNIST':
            exec("model_{} = define_CNN_model({},{},{},{},{},{},{})".format(k,layers,filters,size,padding,pooling,pool_size,pool_stride))
        else:
            exec("model_{} = define_model({},{},'{}')".format(k,layers,nodes,activation))
        exec("model_dict[{}] = model_{}".format(k,k))    
    
    #create folds and fit models to each set of training data; 
    #then calculate residuals on calibration sets
    residual_sets = {}
    kf = KFold(n_splits=folds, shuffle=True, random_state=my_rng)
    counter = 0
    for train_index, cal_index in kf.split(X_train):
        X_train_k, X_cal_k = X_train[train_index], X_train[cal_index]
        y_train_k, y_cal_k = y_train[train_index], y_train[cal_index]
        model_dict[counter].fit(tf.convert_to_tensor(X_train_k), tf.convert_to_tensor(y_train_k), 
                                epochs=my_epochs, batch_size=my_batch, verbose=0)
        y_pred_b = model_dict[counter].predict(tf.convert_to_tensor(X_cal_k), batch_size=len(X_cal_k)).flatten()
        residual_sets[counter] = y_cal_k.flatten() - y_pred_b.flatten()       
        counter+=1

    #predict values in test set using ensemble of fitted networks
    test_preds = np.zeros((len(y_test),folds))
    for b in range(folds):
        test_preds[:,b] = model_dict[b].predict(tf.convert_to_tensor(X_test), batch_size=len(X_test)).flatten()
    ensemble_mean = test_preds.mean(axis=1)
  
    #fit KDE distributions on augmented residual sets and compute p-value of potential residual values
    search_grid = np.linspace(0-search_range/2,
                              0+search_range/2,
                              grid_fineness)
    p_values = np.zeros((len(search_grid),folds,len(trial_bandwidths)))
    for b in range(folds):
        residuals = residual_sets[b]
        for trial, resid_trial in enumerate(search_grid):
            aug_resids = np.concatenate([residuals, np.array([resid_trial])])
            for bandwidth, my_bandwidth in enumerate(trial_bandwidths):
                if my_bandwidth == 0:
                    conform_scores = np.abs(aug_resids)
                else:
                    kde = KernelDensity(bandwidth=my_bandwidth).fit(aug_resids.reshape(-1,1))
                    conform_scores = -kde.score_samples(aug_resids.reshape(-1,1))
                p_values[trial,b,bandwidth] = np.sum(conform_scores <= conform_scores[-1]) / np.size(conform_scores)

    #initialize results array
    results = np.zeros((len(y_test),4,len(trial_bandwidths)))
    results[:,1,:] = y_test.reshape(-1,1)
    results[:,0,:] = test_preds.reshape(-1,1)
    
    #loop over each test value and bandwidth to find and evaluate prediction regions
    for h, my_bandwidth in enumerate(trial_bandwidths):
        mean_p_values = p_values[:,:,h].mean(axis=1)
        pred_region = mean_p_values < CC
        #width
        results[:,3,h] = np.sum(pred_region) / grid_fineness * search_range
        for test_obs, y_true in enumerate(y_test):
            critical_resid = y_true - test_preds[test_obs]
            try:
                lower_arg = np.argwhere(search_grid < critical_resid).flatten().max()
                upper_arg = np.argwhere(search_grid >= critical_resid).flatten().min()
                #in bounds?
                results[test_obs,2,h] = pred_region[lower_arg] * pred_region[upper_arg]
            except:
                #in bounds? 
                results[test_obs,2,h] = 0
        
    return results

In [ ]:
def bootstrap_conformal(X_train, y_train, X_test, y_test, B):
    global my_rng, activation, layers, nodes, size, pooling, padding, filters, pool_size, pool_stride
    global my_epochs, my_batch, my_lr, grid_fineness, search_range, dataset, trial_bandwidths

    #create lists of indices for resampled datasets and their out-of-bag sets
    indices = np.zeros(((len(X_train),resamples)))
    oob_indices = {k: [] for k in range(resamples)}
    for b in range(resamples):
        indices[:, b] = resample(np.arange(len(X_train)), replace=True, n_samples=len(X_train), random_state = my_rng).astype(int)
        oob_indices[b] = np.setdiff1d(np.arange(len(X_train)), indices[:,b],assume_unique=True).astype(int)
        
    #initiate models
    model_dict = {}
    for b in range(B):
        if dataset == 'RotNIST':
            exec("model_{} = define_CNN_model({},{},{},{},{},{},{})".format(k,layers,filters,size,padding,pooling,pool_size,pool_stride))
        else:
            exec("model_{} = define_model({},{},'{}')".format(k,layers,nodes,activation))
        exec("model_dict[{}] = model_{}".format(k,k))    
    
    #fit models to resampled datasets
    for b in range(resamples):
        resampled_X = tf.convert_to_tensor(X_train[indices[:,b].astype(int)])
        resampled_y = tf.convert_to_tensor(y_train[indices[:,b].astype(int)])
        model_b.fit(resampled_X, resampled_y, epochs=my_epochs, batch_size=my_batch, verbose=0)
    
    #identify calibration sets and estimate residuals
    residual_sets = {k: [] for k in range(B)}
    for b in range(B):
        calibration_X_b = tf.convert_to_tensor(X_train[oob_indices[b].astype(int)])
        calibration_y_b = tf.convert_to_tensor(y_train[oob_indices[b].astype(int)])
        y_pred_b = model_dict[b].predict(calibration_X_b, batch_size=len(calibration_X_b)).flatten()
        residual_sets[b] = calibration_y_b.flatten() - y_pred_b.flatten()

    #predict values in test set using ensemble of fitted networks
    test_preds = np.zeros((len(y_test),B))
    for b in range(B):
        test_preds[:,b] = model_dict[b].predict(tf.convert_to_tensor(X_test), batch_size=len(X_test)).flatten()
    ensemble_mean = test_preds.mean(axis=1)
    
    #fit KDE distributions on augmented residual sets and compute p-value of potential residual values
    search_grid = np.linspace(0-search_range/2,
                              0+search_range/2,
                              grid_fineness)
    p_values = np.zeros((len(search_grid),folds,len(trial_bandwidths)))
    for b in range(B):
        residuals = residual_sets[b]
        for trial, resid_trial in enumerate(search_grid):
            aug_resids = np.concatenate([residuals, np.array([resid_trial])])
            for bandwidth, my_bandwidth in enumerate(trial_bandwidths):
                if my_bandwidth == 0:
                    conform_scores = np.abs(aug_resids)
                else:
                    kde = KernelDensity(bandwidth=my_bandwidth).fit(aug_resids.reshape(-1,1))
                    conform_scores = -kde.score_samples(aug_resids.reshape(-1,1))
                p_values[trial,b,bandwidth] = np.sum(conform_scores <= conform_scores[-1]) / np.size(conform_scores)

    #initialize results array
    results = np.zeros((len(y_test),4,len(trial_bandwidths)))
    results[:,1,:] = y_test.reshape(-1,1)
    results[:,0,:] = test_preds.reshape(-1,1)
    
    #loop over each test value and bandwidth to find and evaluate prediction regions
    for h, my_bandwidth in enumerate(trial_bandwidths):
        mean_p_values = p_values[:,:,h].mean(axis=1)
        pred_region = mean_p_values < CC
        #width
        results[:,3,h] = np.sum(pred_region) / grid_fineness * search_range
        for test_obs, y_true in enumerate(y_test):
            critical_resid = y_true - test_preds[test_obs]
            try:
                lower_arg = np.argwhere(search_grid < critical_resid).flatten().max()
                upper_arg = np.argwhere(search_grid >= critical_resid).flatten().min()
                #in bounds?
                results[test_obs,2,h] = pred_region[lower_arg] * pred_region[upper_arg]
            except:
                #in bounds? 
                results[test_obs,2,h] = 0
        
    return results

In [ ]:
#process digits data
#follow instructions at the link below for obtaining the needed files
#https://www.mathworks.com/help/deeplearning/ug/train-a-convolutional-neural-network-for-regression.html

X_train = np.loadtxt('trainX.txt')
X_train = np.reshape(X_train, (28, 5000, 28, 1))
X_train = np.transpose(X_train, (1,0,2, 3))
X_val = np.loadtxt('valX.txt')
X_val = np.reshape(X_val, (28, 5000, 28, 1))
X_val = np.transpose(X_val, (1,0,2, 3))
digits = np.concatenate((X_train, X_val), axis=0)

y_train = np.loadtxt('trainY.txt')
y_train = y_train.reshape(-1,1)
y_val = np.loadtxt('valY.txt')
y_val = y_val.reshape(-1,1)
labels = np.concatenate((y_train, y_val), axis=0)

new_dim = 14

new_digits = np.zeros((10000,new_dim,new_dim,1))

for idx in range(len(digits)):
    img = digits[idx,:,:,:]
    new_img = cv2.resize(img, dsize=(new_dim,new_dim), interpolation=cv2.INTER_LINEAR)
    new_digits[idx,:,:,0] = new_img
    
np.save('digit_images_14', new_digits)
np.save('digit_images_28', digits)
np.save('digit_labels', labels)

In [ ]:
###          Key : [Format, Header_Row, Text_Delimiter, Target_Col, 
###                 Epochs, BatchSize, LearningRate, 
###                 Optimal_Layers, Optimal_Nodes, Optimal_Activation, 
###                 (CNNs: Optimal_layers, Optimal_KernelSize, Optimal_Filters, Optimal_Pooling)
###                 Search Range,
###                 Location (CNNs: input file location, target file location)]
### NOTE: some dataset locations reference a local path; download these datasets first from the links provided in Table 1 (pg 58)
data_dict = {'BostonHousing': [None, None, None, None, 200, 32, 0.001, 3, 100, 'relu', 26.27, None],
             'WineQuality': ['csv',0,';', -1, 75, 32, 0.001, 2, 50, 'relu', 5.12,
                             'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'],             
             'ConcreteStrength': ['excel', 0, None, -1, 200, 32, 0.001, 3, 100, 'relu', 49.67,
                             'https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls'],
             'EnergyEfficiency': ['excel', 0, None, -2, 250, 32, 0.001, 3, 100, 'tanh', 8.49,
                                  'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'],
             'Kinematics': ['csv', 0, ',', -1, 150, 32, 0.001, 3, 75, 'tanh', 0.61,
                            'dataset_2175_kin8nm.csv'],
             'NavalPropulsion': ['csv', 0, ',', 1, 80, 256, 0.001, 3, 100, 'relu', 0.25,
                                 'NavalPropulsion.csv'],
             'PowerPlant': ['csv', 0, ',', -1, 350, 600, 0.01, 3, 50, 'relu', 32.01,
                            'PowerPlant.csv'],
             'ProteinStructure': ['csv', 0, ',', 0, 75, 1024, 0.025, 3, 50, 'tanh', 36.44,
                                  'https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv'],
             'YachtHydrodynamics': ['csv', 0, ',', -1, 500, 64, 0.001, 3, 100, 'relu', 9.12,
                                    'yacht_hydrodynamics.csv'],
             'YearPrediction': ['txt', None, ',', 0, 15, 4096, 0.1, 1, 50, 'relu', 119.83,
                                'YearPredictionMSD.txt'],
             'RotNIST': [None, None, None, None, 20, 64, 1e-3, 3, 5, 32, 'max', 52.34,
                         'digit_images_14.npy', 'digit_labels.npy']}

In [ ]:
def find_data(data_dict, dataset):
    
    file_format = data_dict[dataset][0]
    header_row = data_dict[dataset][1]
    text_delim = data_dict[dataset][2]
    target_col = data_dict[dataset][3]
    my_epochs = data_dict[dataset][4]
    my_batch = data_dict[dataset][5]
    my_lr = data_dict[dataset][6]
    layers = data_dict[dataset][7]
    if dataset == 'RotNIST':
        kernel_size = data_dict[dataset][8]
        filters = data_dict[dataset][9]
        pooling = data_dict[dataset][10]
        search_range = data_dict[dataset][11]
        img_location = data_dict[dataset][12]
        target_location = data_dict[dataset][13]
        case_string = '{}_{}_{}_{}'.format(layers, kernel_size, pooling, filters)
    else:
        nodes = data_dict[dataset][8]
        activation = data_dict[dataset][9]
        search_range = data_dict[dataset][10]
        location = data_dict[dataset][11]
        case_string = '{}_{}_{}'.format(layers, nodes, activation)
    
    if dataset == 'BostonHousing':
        (X,y), (temp,temp2) = keras.datasets.boston_housing.load_data(path="boston_housing.npz", test_split=0, seed=113)
    elif dataset== 'RotNIST':
        X = np.load(img_location)
        y = np.load(target_location)
    else:
        if file_format == 'excel':
            data = pd.read_excel(location, header=header_row)
        else:
            data = pd.read_csv(location, delimiter=text_delim, header=header_row)
        if dataset == 'EnergyEfficiency':
            X = data.iloc[:,:-2].values
            y = data.iloc[:,-2].values
        else:
            y = data.iloc[:,target_col]
            X = data.drop(y.name, axis=1).values
            y = y.values
            
    return X, y, case_string

In [ ]:
###STEP 1 EXPERIMENT

CC = 0.95
trial_bandwidths = [0]
my_rng = np.random.RandomState(seed=2)
test_folds = 5

for dataset in data_dict.keys():
    
    results_dict = {}
    
    X, y, _ = find_data(data_dict, dataset)
    
    if dataset == 'RotNIST':
        filters = 32
        padding = True
        pool_size = 2
        pool_stride = 2
        for layers in [2,3,4]:
            for size in [1,3,5]:
                for pooling in ['max','average']:
                    case_string = '{}_{}_{}'.format(layers, size, pooling)
                    
                    kf = KFold(n_splits=test_folds, shuffle=True, random_state=my_rng)
                    
                    fold = 0
                    for train_index, test_index in kf.split(X):
                        X_train_k, X_test_k = X[train_index], X[test_index]
                        y_train_k, y_test_k = y[train_index], y[test_index]
                        results_dict['bootstrap_1000'] = run_bootstrap(X_train_k, y_train_k, X_test_k, y_test_k, 1000)
                        results_dict['split_conformal'] = split_conformal(X_train_k, y_train_k, X_test_k, y_test_k)

                        for method in results_dict.keys():
                            df = results_dict[method]
                            df.to_csv('Step1_{}_{}_fold{}_{}.csv'.format(dataset,method,fold,case_string))
                        fold += 1
    
    else:
        for layers in [1,2,3]:
            for nodes in [5,10,25,50,75,100]:
                for pooling in ['relu','sigmoid','tanh']:
                    case_string = '{}_{}_{}'.format(layers, nodes, activation)
                    
                    kf = KFold(n_splits=folds, shuffle=True, random_state=my_rng)
                    
                    fold = 0
                    for train_index, test_index in kf.split(X):
                        X_train_k, X_test_k = X[train_index], X[test_index]
                        y_train_k, y_test_k = y[train_index], y[test_index]
                        results_dict['bootstrap_1000'] = run_bootstrap(X_train_k, y_train_k, X_test_k, y_test_k, 1000)
                        results_dict['split_conformal'] = split_conformal(X_train_k, y_train_k, X_test_k, y_test_k)

                        for method in results_dict.keys():
                            my_array = results_dict[method]
                            np.save('Step1_{}_{}_fold{}_{}.npy'.format(dataset,method,fold,case_string), my_array)
                        fold += 1        

In [ ]:
### STEP 2 EXPERIMENT

CC = 0.95
test_size = 100
trial_bandwidths = np.linspace(0,2,21)
my_rng = np.random.RandomState(seed=2)

for dataset in data_dict.keys():
    
    X, y, case_string = find_data(data_dict, dataset)
    
    results_dict = {}
    for trial_seed in range(10):
        test_idx = resample(np.arange(len(X)), replace=False, n_samples = test_size, random_state = trial_seed)
        train_idx = np.setdiff1d(np.arange(len(X)), test_idx, assume_unique=True)
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        if dataset != 'RotNIST':
            X_train, X_test = scale_data(X_train, X_test)
        
        grid_fineness = 1000
        results_dict['bootstrap_100'] = run_bootstrap(X_train, y_train, X_test, y_test, 100)
        results_dict['bootstrap_500'] = run_bootstrap(X_train, y_train, X_test, y_test, 500)
        results_dict['perc_bootstrap_1000'] = run_perc_bootstrap(X_train, y_train, X_test, y_test, 1000)
        results_dict['cross_conformal_5'] = cross_conformal(X_train, y_train, X_test, y_test, 5)
        results_dict['cross_conformal_10'] = cross_conformal(X_train, y_train, X_test, y_test, 10)
        results_dict['cross_conformal_20'] = cross_conformal(X_train, y_train, X_test, y_test, 20)
        results_dict['bootstrap_conformal_5'] = bootstrap_conformal(X_train, y_train, X_test, y_test, 5)
        results_dict['bootstrap_conformal_10'] = bootstrap_conformal(X_train, y_train, X_test, y_test, 10)
        results_dict['bootstrap_conformal_20'] = bootstrap_conformal(X_train, y_train, X_test, y_test, 20)
        results_dict['split_conformal'] = split_conformal(X_train, y_train, X_test, y_test)
        
        grid_fineness = 100
        results_dict['full_conformal'] = run_full_conformal(X_train, y_train, X_test, y_test)
        
        for method in results_dict.keys():
            my_array = results_dict[method]
            np.save('Step2_{}_{}_seed{}_{}.npy'.format(dataset,method,trial_seed,case_string), my_array)